<a href="https://colab.research.google.com/github/mldurga/projects/blob/master/mask_model_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets -Uqq

     |████████████████████████████████| 3.4 MB 10.4 MB/s 
     |████████████████████████████████| 306 kB 50.1 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 3.3 MB 44.2 MB/s 
     |████████████████████████████████| 61 kB 431 kB/s 
     |████████████████████████████████| 895 kB 66.5 MB/s 
     |████████████████████████████████| 243 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 132 kB 57.9 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 
     |████████████████████████████████| 192 kB 54.0 MB/s 
     |████████████████████████████████| 160 kB 61.1 MB/s 


In [2]:
from transformers import AutoModelForMaskedLM

In [3]:
checkpoint="distilbert-base-uncased"
model=AutoModelForMaskedLM.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [6]:
model.num_parameters()/1000000

66.98553

In [7]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
text="this is great [MASK]"

In [10]:
import torch

inputs=tokenizer(text,return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 2307,  103,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [13]:
token_logits=model(**inputs).logits
token_logits

tensor([[[ -5.7670,  -5.7676,  -5.7716,  ...,  -5.1198,  -4.9942,  -3.1208],
         [-12.8342, -12.8560, -12.8926,  ..., -11.4875, -10.8348,  -8.0911],
         [-14.5617, -14.7466, -14.6551,  ..., -12.1280, -10.7909,  -9.2555],
         [-12.2850, -12.5803, -12.4334,  ..., -10.6833, -10.8234,  -6.8367],
         [ -7.6444,  -7.5267,  -7.8744,  ...,  -6.6441,  -7.6132,  -2.1081],
         [-11.5672, -11.5881, -11.5754,  ...,  -9.0213,  -9.5612,  -7.7620]]],
       grad_fn=<AddBackward0>)

In [20]:
token_logits.shape

torch.Size([1, 6, 30522])

In [14]:
tokenizer.mask_token_id

103

In [19]:
mask_token_index=torch.where(inputs['input_ids']==tokenizer.mask_token_id)[1]
mask_token_index

tensor([4])

In [21]:
mask_token_logits=token_logits[0,mask_token_index,:]
mask_token_logits

tensor([[-7.6444, -7.5267, -7.8744,  ..., -6.6441, -7.6132, -2.1081]],
       grad_fn=<IndexBackward0>)

In [32]:
top_5_tokens=torch.topk(mask_token_logits,5,axis=1).indices[0].tolist()

In [33]:
tokenizer.mask_token

'[MASK]'

In [35]:
for i in top_5_tokens:
  print(f'{text.replace(tokenizer.mask_token,tokenizer.decode(i))}')

this is great !
this is great .
this is great beauty
this is great ;
this is great britain


In [36]:
from datasets import load_dataset

In [37]:
imdb=load_dataset('imdb')

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [45]:
imdb['train'].features['label'].names

['neg', 'pos']

In [46]:
sample=imdb['train'].shuffle(seed=42).select(range(3))
sample

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

In [49]:
for x in sample:
  print(x['text'])
  print(x['label'])

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
1
This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds me of S

In [50]:
sample_us=imdb['unsupervised'].shuffle(seed=42).select(range(10))
sample_us

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [ ]:
for x in sample_us:
  print(x['text'])
  print(x['label'])

In [57]:
imdb['test'][23]

{'label': 0,
 'text': 'This movie had a very unique effect on me: it stalled my realization that this movie REALLY sucks! It is disguised as a "thinker\'s film" in the likes of Memento and other jewels like that, but at the end, and even after a few minutes, you come to realize that this is nothing but utter pretentious cr4p. Probably written by some collage student with friends to compassionate to tell him that his writing sucks. The whole idea is \x85 I don\'t even know if it tried to scratch on the supernatural, or they want us to believe that because someone fills your mind (a very weak one, btw) with stupid "riddles", the kind you learn on elementary school recess, you suddenly come to the "one truth" about everything, then you have to kill someone and confess\x85. !!! What? How, what, why, WHY? Is just like saying that to make a cake, just throw a bunch of ingredients, and add water\x85 forgot about cooking it? I guess these guys forgot to, not explain, but present the mechanism 

In [58]:
def tokenize_function(examples):
  result=tokenizer(examples['text'])
  result['word_ids']=[result.word_ids(i) for i in range(len(result['input_ids']))]
  return result

In [60]:
tokenized_datasets=imdb.map(tokenize_function,batched=True,remove_columns=['text','label'])
tokenized_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 50000
    })
})

In [64]:
tokenized_datasets['train'][0].keys()

dict_keys(['attention_mask', 'input_ids', 'word_ids'])

In [68]:
tokenizer.model_max_length

512